In [1]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
from time import sleep

### 1. Define URLs:###

In [2]:
base_2014 = "http://myneta.info/ls2014/"
base_2009 = "http://myneta.info/ls2009/"
base_2004 = "http://myneta.info/loksabha2004/"

#url_list={'2004':ls_2004,'2009':ls_2009,'2014':ls_2014}
base_urls = {'2004':base_2004,'2009':base_2009,'2014':base_2014}

### 2. Define key functions to get data ###

#### The following functions returns a dataframe of constituencies within a state####

In [3]:
def get_constituents(year, thelink):
    html = requests.get(thelink)
    doc = pq(html.content)
    trs = doc('table').eq(2).find('tr')
    datalist=[]
    print ""
    print "Number of constituent: %s :" % len(trs),
    for tr in trs:
        new_state = pq(tr)('th').text()
        if new_state != "":
            print "!",
            curr_state = new_state 
            #print "state: %s" % curr_state
        else:
            print ".",
            districts = pq(tr)('td')
            if districts != "":
                for x in districts:
                    dist_name = pq(x)('a').text()
                    if dist_name != "":
                        district_dict = {'year':year, 'state':curr_state}
                        district_dict['district'] = dist_name
                        district_dict['thelink'] = pq(x)('a').attr.href
                        datalist.append(district_dict)

    return pd.DataFrame(datalist)


#### The following functions returns a dataframe of candidates from a constituency####

In [4]:
def get_candidates(constituency, state, thelink, year):

    masterlink = thelink
    thepage = requests.get(masterlink)
    doc = pq(thepage.content)

    trs = doc('table').eq(2).children('tr')

    #print pq(trs)
    datalist = []

    for tr in trs[1:]:
        candidate_dict = {'Year':year, 'State':state, 'Constituency':constituency, 'Winner': 0}
        for i,td in enumerate(pq(tr)('td')):
            if i == 0:
                candidate_dict['Name'] = pq(td)('a').text()
                candidate_dict['Link'] = pq(td)('a').attr.href
                if pq(td)('font').text() == "Winner":
                    candidate_dict['Winner'] = 1
            if i == 1:
                candidate_dict['Party'] = pq(td).text()
            if i == 2:
                candidate_dict['Criminal_Cases'] = pq(td).text()
            if i == 3:
                candidate_dict['Education'] = pq(td).text()                
            if i == 4:
                candidate_dict['Age'] = pq(td).text() 
            if i == 5:
                candidate_dict['Assets_Rs'] = td.text.replace("Rs",'').replace(',','').lstrip()
            if i == 6:
                candidate_dict['Liabilities_Rs'] = td.text.replace("Rs",'').replace(',','').lstrip()
        datalist.append(candidate_dict)
    return pd.DataFrame(datalist)

### 3. Put it all together and get dataframes###

Get all the contituencies

In [5]:
%%time
constituency_list = []

for year, url in base_urls.iteritems():
    constituency_list.append(get_constituents(year, url))

constituency_df = pd.concat(constituency_list)


Number of constituent: 226 : ! . . ! . . . . . . . . ! . . ! . . . . . ! . . . . . . . . . . . ! . . ! . . . . ! . . ! . . ! . . ! . . . . . . . . ! . . . . ! . . ! . . . ! . . . . . ! . . . . . . . . ! . . . . . . ! . . ! . . . . . . . . . ! . . . . . . . . . . . . . ! . . ! . . ! . . ! . . ! . . . ! . . . . . . . ! . . ! . . . . . ! . . . . . . . . ! . . ! . . . . . . . . . . . ! . . . . . . ! . . ! . . . . . . . . . . . . . . . . . . . . . ! . . . ! . . . . . . . . . . . . 
Number of constituent: 223 : ! . . ! . . . . . . . . . . . . ! . . ! . . . . . ! . . . . . . . . . . . ! . . ! . . . . ! . . ! . . ! . . ! . . . . . . . . ! . . . . ! . . ! . . . ! . . . . . ! . . . . . . . . ! . . . . . . ! . . ! . . . . . . . . . ! . . . . . . . . . . . . . ! . . ! . . ! . . ! . . ! . . . ! . . . . . . . ! . . ! . . . . . ! . . . . . . . . ! . . ! . . . . . . . . . . . ! . . ! . . . . . . . . . . . . . . . . . . . . . ! . . . ! . . . . . . . . . . . . 
Number of constituent: 223 : ! . . ! . . 

Get the constituents for each constituency

In [6]:
%%time
candidates_list = []

for index, row in constituency_df.iterrows() :
    temp_df = get_candidates(row['district'], row['state'], base_urls[row['year']]+row['thelink'], row['year'])
    candidates_list.append(temp_df)
    if index % 10 == 0:
        print ".",
    #sleep(1)
candidates_df = pd.concat(candidates_list)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .CPU times: user 31.4 s, sys: 1.55 s, total: 33 s
Wall time: 8min 49s



In [7]:
print candidates_df.count()
candidates_df[len(candidates_df)-100:len(candidates_df)].head(3)
candidates_df.to_csv('candidates.csv')

Age               19135
Assets_Rs         19135
Constituency      19135
Criminal_Cases    19135
Education         19135
Liabilities_Rs    19135
Link              19135
Name              19135
Party             19135
State             19135
Winner            19135
Year              19135
dtype: int64
